# Product-based Recommendation System
## Based on Moorissa Tjokro tutorial
### Coded by Rebeca Bivar - DB: Armazem Paraíba

### Imports and reading file (data here has only clients, products and purchase count)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import turicreate as tc
from sklearn.model_selection import train_test_split


In [2]:
#READING A FILE WITH ONLY CLIENTS, PRODUCTS BOUGHT AND QTDE
buyers = pd.read_csv('teste.csv', sep=';')
buyers.head()

,COD_CLIENTE,COD_PRODUTO,QUANTIDADE
0,00000301,25243,1
1,00001501,25726,1
2,00002001,21653,1
3,00002701,25303,1
4,00003101,21255,1


##  Data preparation
### Creating dummy table to check if the client has bought a product or not

In [3]:
def create_data_dummy(db):
    data_dummy = db.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

data_dummy = create_data_dummy(buyers)
data_dummy


,COD_CLIENTE,COD_PRODUTO,QUANTIDADE,purchase_dummy
0,00000301,25243,1,1
1,00001501,25726,1,1
2,00002001,21653,1,1
3,00002701,25303,1,1
4,00003101,21255,1,1
...,...,...,...,...
311736,YZ399601,25996,1,1
311737,YZ399801,26327,1,1
311738,YZ400601,26432,1,1
311739,YZ401401,26432,1,1


### Normalizing purchase frequency of each item across users 

In [4]:
#Dummy for marking whether a customer bought that item or not

df_matrix = pd.pivot_table(buyers, values = 'QUANTIDADE', index = 'COD_CLIENTE', columns = 'COD_PRODUTO')


df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

# create a table for input to the modeling  
data_input = df_matrix_norm.reset_index()
data_input.index.names = ['FREQ_COMPRAS']
data_norm = pd.melt(data_input, id_vars=['COD_CLIENTE'], 
                    value_name='FREQ_COMPRAS')

#print(data_norm.shape)
#data_norm.head()

In [5]:
#Just cleaning useless values 
data_norm = data_norm.dropna()
data_norm.head()

,COD_CLIENTE,COD_PRODUTO,FREQ_COMPRAS
1181239,00017101,11332,0.25
1184028,00449301,11332,1.00
1184343,00846001,11332,0.00
1185377,02130901,11332,0.00
1186365,03152901,11332,0.00


### Split data into trainning and testing (80/20)

In [6]:

# Returns train and test datasets as scalable dfs
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

# Now actually splitting purchase_counts, purchase_dummy and purchase_counts_norm
train_data, test_data = split_data(buyers)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## Baseline model to compare and evaluate models


In [7]:
# variables to define field names: 

user_id = 'COD_CLIENTE'
item_id = 'COD_PRODUTO'
users_to_recommend = list(buyers[user_id])
n_recommendation = 10 # itens to recommend
n_display = 30 # display the first few rows in an output dataset

# Function for all models using turicreate
def model(train_data, name, user_id, item_id, target, 
          users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id = user_id, 
                                                    item_id = item_id,
                                                    target = target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

### Popularity Model
   Takes the most popular items for recommendation, which are the products with the highest number of sells across customers.

In [28]:
name = 'popularity'
target = 'QUANTIDADE'
popularity = model(train_data, name, user_id, item_id, target, 
                   users_to_recommend, n_recommendation, n_display)
popularity

Preparing data set.

Data has 249392 observations with 198050 users and 667 items.

Data prepared in: 0.197059s

249392 observations to process; with 667 unique items.

recommendations finished on 1000/311741 queries. users per second: 132820

...

recommendations finished on 307000/311741 queries. users per second: 589358

recommendations finished on 308000/311741 queries. users per second: 584800

recommendations finished on 309000/311741 queries. users per second: 580646

recommendations finished on 310000/311741 queries. users per second: 576356

recommendations finished on 311000/311741 queries. users per second: 572796

In [29]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, 
                   users_to_recommend, n_recommendation, n_display)

Warning: Ignoring columns QUANTIDADE;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 249392 observations with 198014 users and 674 items.

Data prepared in: 0.188017s

...

recommendations finished on 307000/311741 queries. users per second: 610065

recommendations finished on 308000/311741 queries. users per second: 609633

recommendations finished on 309000/311741 queries. users per second: 607555

recommendations finished on 310000/311741 queries. users per second: 603594

recommendations finished on 311000/311741 queries. users per second: 599860

### Using scaled purchase count 

In [30]:
name = 'popularity'
target = 'FREQ_COMPRAS'
pop_norm = model(train_data_norm, name, user_id, item_id, target, 
                 users_to_recommend, n_recommendation, n_display)

Preparing data set.

Data has 245474 observations with 195828 users and 308 items.

Data prepared in: 0.175922s

245474 observations to process; with 308 unique items.

recommendations finished on 1000/311741 queries. users per second: 138850

...

recommendations finished on 307000/311741 queries. users per second: 682533

recommendations finished on 308000/311741 queries. users per second: 678725

recommendations finished on 309000/311741 queries. users per second: 676096

recommendations finished on 310000/311741 queries. users per second: 673544

recommendations finished on 311000/311741 queries. users per second: 670273

### Collaborative Filtering Model

   Recommends items based on how similar clients purchase items. Meaning: if customer 1 and customer 2 bought similar items, for example, 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.
    
   - Lets say X and Y have been rated by costumers 1 and 2. 
   - We then create two item-vectors for both items, then we find the **cosine** or **pearson** distance between these vectors. If the **cosine** value is 1, means total similarity, if it is 0, means no similarity.
   - In this case, we will check the similarity between the target item and other items the customer already bought - using the client's purchase count to items already bought by him as weighing factor (some sort of simulated rating). 

### Using purchase count and purchase frequency
### Cosine

In [31]:
name = 'cosine'
target = 'QUANTIDADE'
cos = model(train_data, name, user_id, item_id, target, 
            users_to_recommend, n_recommendation, n_display)

Preparing data set.

Data has 249392 observations with 198050 users and 667 items.

Data prepared in: 0.190597s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 307000/311741 queries. users per second: 418007

recommendations finished on 308000/311741 queries. users per second: 417711

recommendations finished on 309000/311741 queries. users per second: 416263

recommendations finished on 310000/311741 queries. users per second: 414338

recommendations finished on 311000/311741 queries. users per second: 412055

In [32]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target,
                  users_to_recommend, n_recommendation, n_display)

Warning: Ignoring columns QUANTIDADE;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 249392 observations with 198014 users and 674 items.

Data prepared in: 0.182983s

...

+-------------+-------------+----------------------+------+
| COD_CLIENTE | COD_PRODUTO |        score         | rank |
+-------------+-------------+----------------------+------+
|   00000301  |    21647    |         0.0          |  1   |
|   00000301  |    21255    |         0.0          |  2   |
|   00000301  |    25545    |         0.0          |  3   |
|   00000301  |    24618    |         0.0          |  4   |
|   00000301  |    26181    |         0.0          |  5   |
|   00000301  |    25996    |         0.0          |  6   |
|   00000301  |    25997    |         0.0          |  7   |
|   00000301  |    25354    |         0.0          |  8   |
|   00000301  |    25238    |         0.0          |  9   |
|   00000301  |    19690    |         0.0          |  10  |
|   00001501  |    25727    |  0.0633038878440857  |  1   |
|   00001501  |    25729    | 0.024127662181854248 |  2   |
|   00001501  |    25728    | 0.017444908618927002 |  3   |
|   00001501  |    25983    | 0.01508939

recommendations finished on 154000/311741 queries. users per second: 431120

recommendations finished on 155000/311741 queries. users per second: 431481

recommendations finished on 156000/311741 queries. users per second: 432020

recommendations finished on 157000/311741 queries. users per second: 430492

In [33]:
name = 'cosine'
target = 'FREQ_COMPRAS'
cos_norm = model(train_data_norm, name, user_id, item_id, target, 
                   users_to_recommend, n_recommendation, n_display)


recommendations finished on 158000/311741 queries. users per second: 429502

recommendations finished on 159000/311741 queries. users per second: 429698

recommendations finished on 160000/311741 queries. users per second: 430036

recommendations finished on 161000/311741 queries. users per second: 430511

recommendations finished on 162000/311741 queries. users per second: 431100

...

recommendations finished on 308000/311741 queries. users per second: 503102

recommendations finished on 309000/311741 queries. users per second: 503032

recommendations finished on 310000/311741 queries. users per second: 502663

recommendations finished on 311000/311741 queries. users per second: 501700

+-------------+-------------+------------------------+------+
| COD_CLIENTE | COD_PRODUTO |         score          | rank |
+-------------+-------------+------------------------+------+
|   00000301  |    25737    |          0.0           |  1   |
|   00000301  |    21514    |          0.0           |  2   |
|   00000301  |    25173    |          0.0           |  3   |
|   00000301  |    25354    |          0.0           |  4   |
|   00000301  |    25545    |          0.0           |  5   |
|   00000301  |    26571    |          0.0           |  6   |
|   00000301  |    21653    |          0.0           |  7   |
|   00000301  |    25870    |          0.0           |  8   |
|   00000301  |    26432    |          0.0           |  9   |
|   00000301  |    25700    |          0.0           |  10  |
|   00001501  |    26589    | 0.00017354726791381835 |  1   |
|   00001501  |    24525    | 0.00016473650932312013 |  2   |
|   00001501  |    26588    | 0.0001503455638885498  |  3   |
|   0000

### Using purchase count and purchase frequency
### Pearson

In [34]:
# PURCHASE COUNT
name = 'pearson'
target = 'QUANTIDADE'
pear = model(train_data, name, user_id, item_id, target,
             users_to_recommend, n_recommendation, n_display)

Preparing data set.

Data has 249392 observations with 198050 users and 667 items.

Data prepared in: 0.198606s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 308000/311741 queries. users per second: 346952

recommendations finished on 309000/311741 queries. users per second: 345157

recommendations finished on 310000/311741 queries. users per second: 343772

recommendations finished on 311000/311741 queries. users per second: 341963

+-------------+-------------+--------------------+------+
| COD_CLIENTE | COD_PRODUTO |       score        | rank |
+-------------+-------------+--------------------+------+
|   00000301  |    25099    |        4.0         |  1   |
|   00000301  |    23715    |        2.0         |  2   |
|   00000301  |    21601    |        2.0         |  3   |
|   00000301  |    18941    |        1.5         |  4   |
|   00000301  |    24598    | 1.3714285714285719 |  5   |
|   00000301  |    20503    | 1.3333333333333333 |  6   |
|   00000301  |    22313    |        1.25        |  7   |
|   00000301  |    20796    | 1.2142857142857142 |  8   |
|   00000301  |    24619    |        1.2         |  9   |
|   00000301  |    23575    |        1.2         |  10  |
|   00001501  |    25099    |        4.0         |  1   |
|   00001501  |    23715    |        2.0         |  2   |
|   00001501  |    21601    |        2.0         |  3   |
|   00001501  |    18941    |        1.5         |  4   |
|   00001501  

In [35]:
# PURCHASE DUMMY
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target,
                   users_to_recommend, n_recommendation, n_display)

Warning: Ignoring columns QUANTIDADE;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 249392 observations with 198014 users and 674 items.

Data prepared in: 0.175367s

...

recommendations finished on 308000/311741 queries. users per second: 367950

recommendations finished on 309000/311741 queries. users per second: 367340

recommendations finished on 310000/311741 queries. users per second: 365157

recommendations finished on 311000/311741 queries. users per second: 363023

+-------------+-------------+-------+------+
| COD_CLIENTE | COD_PRODUTO | score | rank |
+-------------+-------------+-------+------+
|   00000301  |    21647    |  0.0  |  1   |
|   00000301  |    21255    |  0.0  |  2   |
|   00000301  |    25545    |  0.0  |  3   |
|   00000301  |    24618    |  0.0  |  4   |
|   00000301  |    26181    |  0.0  |  5   |
|   00000301  |    25996    |  0.0  |  6   |
|   00000301  |    25997    |  0.0  |  7   |
|   00000301  |    25354    |  0.0  |  8   |
|   00000301  |    25238    |  0.0  |  9   |
|   00000301  |    19690    |  0.0  |  10  |
|   00001501  |    21647    |  0.0  |  1   |
|   00001501  |    21255    |  0.0  |  2   |
|   00001501  |    25545    |  0.0  |  3   |
|   00001501  |    24618    |  0.0  |  4   |
|   00001501  |    26181    |  0.0  |  5   |
|   00001501  |    25996    |  0.0  |  6   |
|   00001501  |    25997    |  0.0  |  7   |
|   00001501  |    25354    |  0.0  |  8   |
|   00001501  |    25238    |  0.0  |  9   |
|   000015

In [36]:
# PURCHASE FREQUENCY
name = 'pearson'
target = 'FREQ_COMPRAS'
pear_norm = model(train_data_norm, name, user_id, item_id, target,
                  users_to_recommend, n_recommendation, n_display)

Preparing data set.

Data has 245474 observations with 195828 users and 308 items.

Data prepared in: 0.16542s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 308000/311741 queries. users per second: 381631

recommendations finished on 309000/311741 queries. users per second: 376208

recommendations finished on 310000/311741 queries. users per second: 372416

recommendations finished on 311000/311741 queries. users per second: 369026

+-------------+-------------+---------------------+------+
| COD_CLIENTE | COD_PRODUTO |        score        | rank |
+-------------+-------------+---------------------+------+
|   00000301  |    21126    |  0.3333333333333333 |  1   |
|   00000301  |    21602    |  0.3333333333333333 |  2   |
|   00000301  |    25418    |         0.2         |  3   |
|   00000301  |    22313    |         0.2         |  4   |
|   00000301  |    24619    |         0.2         |  5   |
|   00000301  |    20796    | 0.18181818181818182 |  6   |
|   00000301  |    24392    | 0.14285714285714285 |  7   |
|   00000301  |    21600    | 0.12500000000000003 |  8   |
|   00000301  |    23575    |        0.125        |  9   |
|   00000301  |    20490    |  0.1111111111111111 |  10  |
|   00001501  |    21126    |  0.3333333333333333 |  1   |
|   00001501  |    21602    |  0.3333333333333333 |  2   |
|   00001501  |    25418    |         0.2         |  3   |
|   00001501  |    22313    |         0.2         |  4  

## Model Evaluation 

### RMSE - Root Mean Squared Errors

   - Measures the error of predicted values
   - Lesser the RMSE values, better the recommendations
   
### Precision-Recall

   - Recall: Percentage of products that a customer buys that are actually recommended. 
   - Precision: How many itens the customer liked out of the recommended?
   - The idea is to optimze both recall and precision to be close as 1 as possible


In [37]:
# Variables for model evaluation

models_counts = [popularity, cos, pear]
models_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_norm = [pop_norm, cos_norm, pear_norm]

names_counts = ['Popularity Model on Purchase Counts', 
                  'Cosine Similarity on Purchase Counts', 
                  'Pearson Similarity on Purchase Counts']
names_dummy = ['Popularity Model on Purchase Dummy', 
                 'Cosine Similarity on Purchase Dummy', 
                 'Pearson Similarity on Purchase Dummy']
names_norm = ['Popularity Model on Scaled Purchase Counts', 
                'Cosine Similarity on Scaled Purchase Counts', 
                'Pearson Similarity on Scaled Purchase Counts']


eval_counts = tc.recommender.util.compare_models(test_data, models_counts, 
                                                 model_names=names_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_dummy,
                                                model_names=names_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_norm, 
                                               model_names=names_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/58261 queries. users per second: 107285

recommendations finished on 2000/58261 queries. users per second: 149220

recommendations finished on 3000/58261 queries. users per second: 173752

recommendations finished on 4000/58261 queries. users per second: 192382

...

recommendations finished on 54000/57311 queries. users per second: 132364

recommendations finished on 55000/57311 queries. users per second: 131372

recommendations finished on 56000/57311 queries. users per second: 130550

recommendations finished on 57000/57311 queries. users per second: 125097


Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    | 1.7448657325818782e-05 | 1.7448657325818782e-05 |
|   2    | 8.724328662909391e-06  | 1.7448657325818782e-05 |
|   3    | 2.908109554303127e-05  | 7.851895796618452e-05  |
|   4    |  3.05351503201828e-05  |  0.000113416272617822  |
|   5    | 2.791785172131007e-05  | 0.00013086492994364066 |
|   6    |  3.19892050973344e-05  | 0.0001832109019210974  |
|   7    | 3.738997998389731e-05  |  0.000253005531224372  |
|   8    | 5.2345971977456394e-05 | 0.0004013191184938318  |
|   9    | 5.040723227458754e-05  | 0.0004362164331454697  |
|   10   | 4.711137477971077e-05  | 0.00045366509047128694 |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0791320499141185

Per User RMSE (best)
+-------------+-